# Road Following - Live demo

In this notebook, we will use model we trained to move jetBot smoothly on track. 

### Load Trained Model

We will assume that you have already downloaded ``best_steering_model_xy.pth`` to work station as instructed in "train_model.ipynb" notebook. Now, you should upload model file to JetBot in to this notebook's directory. Once that's finished there should be a file named ``best_steering_model_xy_model.pth`` in this notebook's directory.

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model. This should look very familiar from the training notebook.

In [ ]:
# %matplotlib notebook
from jetbot import RoadCruiser

model = 'resnet18' # resnet18, resnet34, resnet50, mobilenet_v3_large
type_model = 'resnet', # resnet, mobilenet
RC = RoadCruiser(cruiser_model=model, type_cruiser_model=type_model)

Next, load the trained weights from the ``best_steering_model_xy_model.pth`` file that you uploaded.

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

### Creating the Pre-Processing Function

We have now loaded our model, but there's a slight issue. The format that we trained our model doesn't exactly match the format of the camera. To do that, we need to do some preprocessing. This involves the following steps:

1. Convert from HWC layout to CHW layout
2. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
3. Transfer the data from CPU memory to GPU memory
4. Add a batch dimension

Awesome! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera. You should be pretty familiar with this by now. 

In [ ]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import bgr8_to_jpeg

image_widget = ipywidgets.Image()
fps_widget = ipywidgets.FloatText(description='Capture rate')

traitlets.dlink((RC.camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((RC.camera, 'cap_time'), (fps_widget, 'value'))
                

Now, we will define sliders to control JetBot
> Note: We have initialize the slider values for best known configurations, however these might not work for your dataset, therefore please increase or decrease the sliders according to your setup and environment

1. Speed Control (speed_gain_slider): To start your JetBot increase ``speed_gain_slider`` 
2. Steering Gain Control (steering_gain_slider): If you see JetBot is wobbling, you need to reduce ``steering_gain_slider`` till it is smooth
3. Steering Bias control (steering_bias_slider): If you see JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets

> Note: You should play around above mentioned sliders with lower speed to get smooth JetBot road following behavior.

In [ ]:
speed_gain_slider = ipywidgets.FloatSlider(min=0, max=1, step=0.001, value=0.2, description='speed gain', readout_format='.3f')
steering_gain_slider = ipywidgets.FloatSlider(min=0, max=0.5, step=0.001, value=0.08, description='steering gain', readout_format='.3f')
steering_dgain_slider = ipywidgets.FloatSlider(min=0, max=2.0, step=0.001, value=0.82, description='steering kd', readout_format='.3f')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.1, max=0.1, step=0.001, value=-0.01, description='steering bias', readout_format='.3f')

traitlets.dlink((speed_gain_slider, 'value'), (RC, 'speed_gain'))
traitlets.dlink((steering_gain_slider, 'value'), (RC, 'steering_gain'))
traitlets.dlink((steering_dgain_slider, 'value'), (RC, 'steering_dgain'))
traitlets.dlink((steering_bias_slider, 'value'), (RC, 'steering_bias'))

VBox_image = ipywidgets.VBox([image_widget, fps_widget], layout=ipywidgets.Layout(align_self='center'))
VBox_control = ipywidgets.VBox([speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider], layout=ipywidgets.Layout(align_self='center'))
# display(ipywidgets.HBox([VBox_image, VBox_control]))

Next, let's display some sliders that will let us see what JetBot is thinking.  The x and y sliders will display the predicted x, y values.

The steering slider will display our estimated steering value.  Please remember, this value isn't the actual angle of the target, but simply a value that is
nearly proportional.  When the actual angle is ``0``, this will be zero, and it will increase / decrease with the actual angle.  

In [ ]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

traitlets.dlink((RC, 'x_slider'), (x_slider, 'value'))
traitlets.dlink((RC, 'y_slider'), (y_slider, 'value'))
traitlets.dlink((RC, 'steering'), (steering_slider, 'value'))
traitlets.dlink((RC, 'speed'), (speed_slider, 'value'))

Box_y_state = ipywidgets.HBox([y_slider, speed_slider])
Box_x_state = ipywidgets.VBox([x_slider, steering_slider])
Box_state = ipywidgets.VBox([Box_y_state, Box_x_state])
# display(ipywidgets.HBox([y_slider, speed_slider]))
# display(x_slider, steering_slider)

Next, we'll create a function that will get called whenever the camera's value changes. This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. Compute the approximate steering value
4. Control the motors using proportional / derivative control (PD)

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

>WARNING: This code will move the robot!! Please make sure your robot has clearance and it is on Lego or Track you have collected data on. The road follower should work, but the neural network is only as good as the data it's trained on!

In [ ]:
RC.start_cruising()

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame. 

You can now place JetBot on  Lego or Track you have collected data on and see whether it can follow track.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [ ]:
display(ipywidgets.HBox([Box_state, VBox_image, VBox_control]))

button_stop = ipywidgets.Button(description='Stop', tooltip='Click to stop running',icon='fa-circle-stop')
display(button_stop)
button_stop.on_click(RC.stop_cruising)

Again, let's close the camera conneciton properly so that we can use the camera in other notebooks.

### Conclusion
That's it for this live demo! Hopefully you had some fun seeing your JetBot moving smoothly on track following the road!!!

If your JetBot wasn't following road very well, try to spot where it fails. The beauty is that we can collect more data for these failure scenarios and the JetBot should get even better :)